# GTF GFF3 PARSER
########################################################################
## 05/19/2018
## By Xiang Li,
## lux@gwu.edu
## Peng's Lab1
## Ver.1.0
########################################################################


In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def DIR_CHECK_CREATE(Input_Path):
    if (not os.path.isdir(Input_Path)):
        print ("New Dir Made is" + Input_Path)
        os.mkdir(Input_Path)
    else: 
        print ('Dir Exists')
def Add_common_header(df,common_header):
    name_list=[common_header]*len(df.columns)
    tuples = list(zip(name_list,df.columns))
    df.columns = pd.MultiIndex.from_tuples(tuples)
    return df

In [159]:
### gtf
def Get_Simple_Genes_gtf2bed(df, out_name):

    df_bed = []
    i=0
    ### Split last column, (gene_id, transcript_id and etc)
    df = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')
    ### Group by gene_id and transcript_id,
    ### If counting results are equal, means it is a simple gene.
    ### Some wierd loopole, some row does not have p_id (col 11), some has
    ### without p_id, then 10 is transcript_id
    df_no_p_id = df[df[12]=='']
    ### with p_id, then 11 is transcript_id
    df_with_p_id = df[df[12]!='']

    ### Iterating Each Gene Group and convert to bed format.
    for gene_id, gene_group in df_no_p_id.groupby([8]):
        if (len(gene_group[10].unique()) == 1):
            df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6]])
            i+=1
    for gene_id, gene_group in df_with_p_id.groupby([8]):
        if (len(gene_group[11].unique()) == 1):
            df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6]])
            i+=1
    ### Output        
    df_bed = pd.DataFrame(df_bed, columns=['#chr','start','end','name','score','strand']).sort_values(by=['#chr','start'])
    df_bed.to_csv(str(i)+'_'+out_name+'_Simple_Gene_list.bed', sep='\t', index=None)
    return df_bed

def Get_Genes_gtf2bed(df, out_name):
    df_bed = []
    i=0
    ### Split last column, (gene_id, transcript_id and etc)
    df = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')
      
    ### Iterating Each Gene Group and convert to bed format.
    for gene_id, gene_group in df.groupby([8]):
        df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6]])
        i+=1
    ### Output        
    df_bed = pd.DataFrame(df_bed, columns=['#chr','start','end','name','score','strand']).sort_values(by=['#chr','start'])
    df_bed.to_csv(str(i)+'_'+out_name+'_Gene_list.bed', sep='\t', index=None)
    return df_bed


In [160]:
PATH_FOLDER=os.getcwd()
DIR_CHECK_CREATE(PATH_FOLDER)
INPUT_LIST=os.listdir(PATH_FOLDER)

NAME = '/hg19_2015.gtf'
NAME2= '/hg19_2015.gff3'

df = pd.read_csv(PATH_FOLDER+NAME, sep='\t',skiprows=None, header=None ) #, nrows=200)
#df = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')

#df2 = pd.read_csv(PATH_FOLDER+NAME2, sep='\t',skiprows=2, header=None , nrows=200)
#df2 = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')

#df
xx = Get_Simple_Genes_gtf2bed(df, 'hg19_2015')

Dir Exists


In [140]:
df_no_p_id = df[df[12]=='']
### with p_id, then 11 is transcript_id
df_with_p_id = df[df[12]!='']
